In [84]:
import pandas as pd
import geopandas as gpd

In [85]:
# Panda settings for showing data (this is foremost done to more easily explore the data while processing it)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [86]:
input_folder = "E://bomen_data//input//"
output_folder = "E://bomen_data//output//"

Translating the inspection data into English

In [87]:
tree_inspections = pd.read_csv(input_folder+'bomen-veiligheidsinspecties-2024-11.csv')

In [ ]:
tree_inspections['Advice'] = tree_inspections['Snoeiwijze'].replace({
    'Begeleidingssnoei': 'Growth anticipating pruning',
    'Onderhoudssnoei': 'maintenance pruning', # height 18m + & Trunk diameter 0,5 meter + IS high risk 
    'Knotten': 'Pollarding' ,
    'Fruitsnoei': 'Fruit pruning',
    'Terugzetten (extensief)': 'extensive topiary tree pruning',
    'Niet van toepassing': 'Not applicable',
    'Terugzetten (intensief)': 'intensive topiary tree pruning',
    'Scheren | knippen': 'Shaving | cutting',
    'Kandelaberen': 'Topping', # high risk
    'Innemen kroon' : 'Reduding crown' # high risk
})

In [89]:
tree_inspections.rename(columns={'Boomid': 'TreeId'}, inplace=True)
tree_inspections.rename(columns={'Inspecterendeorganisatie': 'Inspecting_company'}, inplace=True)
tree_inspections.rename(columns={'Inspectiedatum': 'Inspection_date'}, inplace=True)

In [90]:
selected_columns = ['Id', 'TreeId','Inspecting_company','Inspection_date','Advice']
tree_inspections.to_csv(output_folder+'tree_inspections_11_2024.csv', columns=selected_columns, index=False)

Translating the tree data of Amsterdam from november 2024 data into English: source -> https://api.data.amsterdam.nl/v1/bomen/stamgegevens/?_format=geojson (nov 2024)

In [91]:
tree_data_memory_nov24 = gpd.read_file(input_folder+'bomen-stamgegevens_2024_11.json')


In [92]:
tree_data_nov24 = tree_data_memory_nov24 

In [93]:
tree_data_nov24.head(5)

,id,gbdBuurtId,typeBeheerderPlus,boomhoogteklasseActueel,typeEigenaarPlus,jaarVanAanleg,soortnaam,stamdiameterklasse,standplaats,standplaatsGedetailleerd,typeObject,typeSoortnaam,soortnaamKort,soortnaamTop,geometry
0,919933,03630980000301,R&E_VOR_Bomen,e. 15 tot 18 m.,Gemeente Amsterdam,1948.0,Tilia americana,None,None,None,Boom niet vrij uitgroeiend,Bomen,Tilia,Linde (Tilia),POINT (4.90467 52.33981)
1,919934,03630980000301,R&E_VOR_Bomen,c. 9 tot 12 m.,Gemeente Amsterdam,1978.0,Ulmus hollandica 'Vegeta',None,Elementenverharding,Tegels,Boom niet vrij uitgroeiend,Bomen,Ulmus,Iep (Ulmus),POINT (4.90269 52.34009)
2,919935,03630980000311,R&E_VOR_Bomen,c. 9 tot 12 m.,Gemeente Amsterdam,1990.0,Fraxinus excelsior 'Westhof's Glorie',"0,2 tot 0,3 m.",None,None,Boom niet vrij uitgroeiend,Bomen,Fraxinus,Es (Fraxinus),POINT (4.85521 52.33198)
3,919936,03630980000297,R&E_VOR_Bomen,b. 6 tot 9 m.,Gemeente Amsterdam,2002.0,Ulmus glabra 'Lutescens',None,Heesters,Fijne sierheester,Boom niet vrij uitgroeiend,Bomen,Ulmus,Iep (Ulmus),POINT (4.90368 52.34883)
4,919937,03630980000306,R&E_VOR_Bomen,b. 6 tot 9 m.,Gemeente Amsterdam,1985.0,Quercus robur,None,Bosplantsoen,Struikvormers,Boom niet vrij uitgroeiend,Bomen,Quercus,Eik (Quercus),POINT (4.87589 52.34106)


In [94]:
diameter_trunks = tree_data_nov24['stamdiameterklasse'].unique()
diameter_trunks

array([None, '0,2 tot 0,3 m.', '0,1 tot 0,2 m.', '0,3 tot 0,5 m.',
       '0,5 tot 1 m.', '1,0 tot 1,5 m.', 'Onbekend', '1,5 m. en groter',
       'tot 0,1 m.'], dtype=object)

In [95]:
tree_data_nov24['diameter_tree_trunk'] = tree_data_nov24['stamdiameterklasse'].replace({
     '0,2 tot 0,3 m.': "0.2 to 0.3 m.",
     '0,1 tot 0,2 m.': "0.1 to 0.2 m.",
     '0,3 tot 0,5 m.': "0.3 to 0.5 m.",
     '0,5 tot 1 m.': "0.5 to 1 m.",
     '1,0 tot 1,5 m.':"1.0 to 1.5 m.",
     'Onbekend': "unknown",
     '1,5 m. en groter': "1.5 m. and bigger",
     'tot 0,1 m.': "smaller than 0.1 m."
})

In [96]:
tree_heigth = tree_data_nov24['boomhoogteklasseActueel'].unique()
tree_heigth

array(['e. 15 tot 18 m.', 'c. 9 tot 12 m.', 'b. 6 tot 9 m.',
       'd. 12 tot 15 m.', 'a. tot 6 m.', 'f. 18 tot 24 m.',
       'q. Niet van toepassing', 'g. 24 m. en hoger',
       'p. Niet te beoordelen', None,
       'r. Niet van toepassing (te vellen boom)',
       'ZZ_Vervallen 6 tot 12 m.', 'ZZ_Vervallen 12 tot 18 m.'],
      dtype=object)

In [97]:
tree_data_nov24['height_of_tree'] = tree_data_nov24['boomhoogteklasseActueel'].replace({
     'e. 15 tot 18 m.': 'e. 15 to 18 ',
     'c. 9 tot 12 m.': 'c. 9 to 12 m.' ,
     'b. 6 tot 9 m.': 'b. 6 to 9 m.' ,
     'd. 12 tot 15 m.': 'd. 12 to 15 m.',
     'a. tot 6 m.': 'a. to 6 m.' ,
     'f. 18 tot 24 m.': 'f. 18 to 24 m.',
     'q. Niet van toepassing': 'q. Not applicable',
     'g. 24 m. en hoger': 'g. 24 m. and higher',
     'p. Niet te beoordelen': 'p. Impossible to determine'  ,
     'r. Niet van toepassing (te vellen boom)': 'r. Not applicabel (tree need to be removed)' ,
     'ZZ_Vervallen 6 tot 12 m.': 'ZZ_Discard 6 to 12 m.' ,
     'ZZ_Vervallen 12 tot 18 m.': 'ZZ_Discard 12 to 18 m.'
})

In [98]:
tree_heigth = tree_data_nov24['typeObject'].unique()
tree_heigth

array(['Boom niet vrij uitgroeiend', 'Boom vrij uitgroeiend', 'Stobbe',
       'Gekandelaberde boom', 'Knotboom', 'Vormboom', 'Fruitboom',
       'Leiboom', None, 'Bosplantsoen'], dtype=object)

In [99]:
tree_data_nov24['object_type'] = tree_data_nov24['typeObject'].replace({
     'Boom niet vrij uitgroeiend': 'non free growing tree',
     'Boom vrij uitgroeiend':'free growing tree',
     'Stobbe':'Tree Stub',
     'Gekandelaberde boom': 'Topped tree',
     'Knotboom': 'Pollarded tree',
     'Vormboom': 'Topiary tree',
     'Fruitboom': 'Fruit tree',
     'Leiboom':'Espalier tree',
     'Bosplantsoen': 'Forest park',
     'Aanwijzingsteken' : 'Indication sign'

})

In [100]:
tree_data_nov24.head(5)

,id,gbdBuurtId,typeBeheerderPlus,boomhoogteklasseActueel,typeEigenaarPlus,jaarVanAanleg,soortnaam,stamdiameterklasse,standplaats,standplaatsGedetailleerd,typeObject,typeSoortnaam,soortnaamKort,soortnaamTop,geometry,diameter_tree_trunk,height_of_tree,object_type
0,919933,03630980000301,R&E_VOR_Bomen,e. 15 tot 18 m.,Gemeente Amsterdam,1948.0,Tilia americana,None,None,None,Boom niet vrij uitgroeiend,Bomen,Tilia,Linde (Tilia),POINT (4.90467 52.33981),None,e. 15 to 18,non free growing tree
1,919934,03630980000301,R&E_VOR_Bomen,c. 9 tot 12 m.,Gemeente Amsterdam,1978.0,Ulmus hollandica 'Vegeta',None,Elementenverharding,Tegels,Boom niet vrij uitgroeiend,Bomen,Ulmus,Iep (Ulmus),POINT (4.90269 52.34009),None,c. 9 to 12 m.,non free growing tree
2,919935,03630980000311,R&E_VOR_Bomen,c. 9 tot 12 m.,Gemeente Amsterdam,1990.0,Fraxinus excelsior 'Westhof's Glorie',"0,2 tot 0,3 m.",None,None,Boom niet vrij uitgroeiend,Bomen,Fraxinus,Es (Fraxinus),POINT (4.85521 52.33198),0.2 to 0.3 m.,c. 9 to 12 m.,non free growing tree
3,919936,03630980000297,R&E_VOR_Bomen,b. 6 tot 9 m.,Gemeente Amsterdam,2002.0,Ulmus glabra 'Lutescens',None,Heesters,Fijne sierheester,Boom niet vrij uitgroeiend,Bomen,Ulmus,Iep (Ulmus),POINT (4.90368 52.34883),None,b. 6 to 9 m.,non free growing tree
4,919937,03630980000306,R&E_VOR_Bomen,b. 6 tot 9 m.,Gemeente Amsterdam,1985.0,Quercus robur,None,Bosplantsoen,Struikvormers,Boom niet vrij uitgroeiend,Bomen,Quercus,Eik (Quercus),POINT (4.87589 52.34106),None,b. 6 to 9 m.,non free growing tree


In [101]:
tree_data_nov24.rename(columns={'typeEigenaarPlus': 'maintainer'}, inplace=True) #For educational purposes we parsed this to a new field called maintainer instead of owner
tree_data_nov24.rename(columns={'jaarVanAanleg': 'year_planted'}, inplace=True)
tree_data_nov24.rename(columns={'soortnaam': 'botanical_name'}, inplace=True)
tree_data_nov24.rename(columns={'soortnaamTop': 'name'}, inplace=True)
tree_data_nov24.rename(columns={'soortnaam': 'botanical_name'}, inplace=True)

tree_data_nov24 = tree_data_nov24.drop(columns=['gbdBuurtId','typeObject','boomhoogteklasseActueel','stamdiameterklasse','typeBeheerderPlus','standplaats', 'standplaatsGedetailleerd', 'typeSoortnaam','soortnaamKort'])

In [102]:
tree_data_nov24.to_file(output_folder+'trees_nov_2024.geojson', driver='GeoJSON')

Translating the tree data from October 2024 data into English obtained from https://maps.amsterdam.nl/open_geodata/ (currently, nov 2024 not available anymore)

In [103]:
tree_data_memory_oct24 = gpd.read_file(input_folder+'Trees_amsterdam_2024_10.gpkg')


In [104]:
tree_data_oct24 =tree_data_memory_oct24

In [105]:
tree_data_oct24.head()

,Boomnummer,Soortnaam,SoortnaamKortNL,SoortnaamTop15,Jaar_van_aanleg,Boomhoogte,Type_detail,Standplaats_detail,Beheerder,geometry
0,1106188,Ulmus hollandica `Vegeta`,Hollandse iep,Iep,1981,18 tot 24 m.,Boom niet vrij uitgroeiend,Gras- en kruidachtigen - Gazon,Gemeente,POINT (4.82107 52.34210)
1,1106189,Fraxinus excelsior `Westhof`s Glorie`,Gewone es,Es,1981,18 tot 24 m.,Boom niet vrij uitgroeiend,Gras- en kruidachtigen - Gazon,Gemeente,POINT (4.82122 52.34211)
2,1106190,Ulmus hollandica `Vegeta`,Hollandse iep,Iep,1981,18 tot 24 m.,Boom niet vrij uitgroeiend,Gras- en kruidachtigen - Gazon,Gemeente,POINT (4.82127 52.34212)
3,1106191,Fraxinus excelsior cv.,Gewone es,Es,1981,12 tot 15 m.,Boom niet vrij uitgroeiend,Gras- en kruidachtigen - Ruw gras,Gemeente,POINT (4.82462 52.34227)
4,1106192,Ulmus hollandica `Vegeta`,Hollandse iep,Iep,1981,15 tot 18 m.,Boom niet vrij uitgroeiend,Gras- en kruidachtigen - Ruw gras,Gemeente,POINT (4.82454 52.34226)


In [106]:
tree_data_oct24['current_height_of_tree'] = tree_data_oct24['Boomhoogte'].replace({
     '15 tot 18 m.': '15 to 18 ',
     '9 tot 12 m.': '9 to 12 m.' ,
     '6 tot 9 m.': '6 to 9 m.' ,
     '12 tot 15 m.': '12 to 15 m.',
     'tot 6 m.': 'to 6 m.' ,
     '18 tot 24 m.': '18 to 24 m.',
     'Niet van toepassing': 'Not applicable',
     '24 m. en hoger': '24 m. and higher',
     'Niet te beoordelen': 'Impossible to determine'  ,
     'Niet van toepassing (te vellen': 'Not applicabel (tree need to be removed)' ,
     'Vervallen 6 tot 12 m.': 'Discard 6 to 12 m.' ,
     'Vervallen 12 tot 18 m.': 'Discard 12 to 18 m.'
})

In [107]:
# https://www.vdberk.com/ has been used as a source to translate Dutch into English

tree_data_oct24['type_detail'] = tree_data_oct24['Type_detail'].replace({
    'Boom niet vrij uitgroeiend':'non free growing tree', 
    'Boom vrij uitgroeiend':'free growing tree',
    'Knotboom':'Pollarded tree',
    'Vormboom - Blokboom':'Topiary tree - block', 
    'Gekandelaberde boom':'Topped tree',
    'Fruitboom - Laagstam':'Fruit tree - halfstem', 
    'Vormboom - Bolboom':'Topiary tree - sphere',
    'Vormboom - Geschoren boom':'Topiary tree - shaved', 
    'Vormboom': 'Topiary tree',
    'Fruitboom': 'Fruit tree',
    'Vormboom - Dakboom':'Topiary tree - roof', 
    'Fruitboom - Hoogstam':'Fruit tree - clearstem', 
    'Leiboom - Geknipte leiboom':'Espalier tree',
    'Boom niet vrij uitgroeiend - Bolboo': 'free growing tree', 
    'Leiboom':'Espalier tree',
    'Bosplantsoen - Boomvormers':'Forest park'
})

In [108]:
tree_data_oct24.head()

,Boomnummer,Soortnaam,SoortnaamKortNL,SoortnaamTop15,Jaar_van_aanleg,Boomhoogte,Type_detail,Standplaats_detail,Beheerder,geometry,current_height_of_tree,type_detail
0,1106188,Ulmus hollandica `Vegeta`,Hollandse iep,Iep,1981,18 tot 24 m.,Boom niet vrij uitgroeiend,Gras- en kruidachtigen - Gazon,Gemeente,POINT (4.82107 52.34210),18 to 24 m.,non free growing tree
1,1106189,Fraxinus excelsior `Westhof`s Glorie`,Gewone es,Es,1981,18 tot 24 m.,Boom niet vrij uitgroeiend,Gras- en kruidachtigen - Gazon,Gemeente,POINT (4.82122 52.34211),18 to 24 m.,non free growing tree
2,1106190,Ulmus hollandica `Vegeta`,Hollandse iep,Iep,1981,18 tot 24 m.,Boom niet vrij uitgroeiend,Gras- en kruidachtigen - Gazon,Gemeente,POINT (4.82127 52.34212),18 to 24 m.,non free growing tree
3,1106191,Fraxinus excelsior cv.,Gewone es,Es,1981,12 tot 15 m.,Boom niet vrij uitgroeiend,Gras- en kruidachtigen - Ruw gras,Gemeente,POINT (4.82462 52.34227),12 to 15 m.,non free growing tree
4,1106192,Ulmus hollandica `Vegeta`,Hollandse iep,Iep,1981,15 tot 18 m.,Boom niet vrij uitgroeiend,Gras- en kruidachtigen - Ruw gras,Gemeente,POINT (4.82454 52.34226),15 to 18,non free growing tree


In [109]:
tree_data_oct24['Beheerder'] = tree_data_oct24['Beheerder'].replace({
    'Gemeente':'Gemeente Amsterdam'}) 

In [110]:
tree_data_oct24.rename(columns={'Boomnummer': 'tree_number'}, inplace=True)
tree_data_oct24.rename(columns={'Soortnaam': 'botanical_name'}, inplace=True)
tree_data_oct24.rename(columns={'SoortnaamKortNL': 'Dutch_name'}, inplace=True)
tree_data_oct24.rename(columns={'Jaar_van_aanleg': 'year_planted'}, inplace=True)
tree_data_oct24.rename(columns={'Beheerder': 'maintainer'}, inplace=True)

tree_data_oct24 = tree_data_oct24.drop(columns=['SoortnaamTop15','Type_detail','Standplaats_detail', 'Boomhoogte'])

In [111]:
tree_data_oct24.to_file(output_folder+'tree_data_oct24.geojson', layer='oct_2024', driver="GPKG")

Translating the tree data from October 2023 data into English obtained from https://maps.amsterdam.nl/open_geodata/ (currently, nov 2024 not available anymore)

In [112]:
tree_data_memory_oct23 = gpd.read_file(input_folder+'Trees_Amsterdam_2023_09.shp')

In [113]:
tree_data_oct23 = tree_data_memory_oct23

In [114]:
tree_data_oct23.head()

,id,soortnaam,soortnaamK,SoortnaamN,boomhoogte,stamdiamet,jaarVanAan,typeObject,geometry
0,919934,Ulmus hollandica `Vegeta`,Ulmus,Huntingdon-iep,b. 6 tot 9 m.,None,1978,Boom niet vrij uitgroeiend,POINT (121980.449 483684.998)
1,919935,Fraxinus excelsior `Westhof`s Glorie`,Fraxinus,Gewone es (cultuurvariëteit),c. 9 tot 12 m.,"0,2 tot 0,3 m.",1990,Boom niet vrij uitgroeiend,POINT (118737.842 482805.464)
2,919936,Ulmus glabra `Lutescens`,Ulmus,Gele bergiep (cultuurvariëteit),b. 6 tot 9 m.,None,2002,Boom niet vrij uitgroeiend,POINT (122053.589 484657.432)
3,919937,Quercus robur,Quercus,Zomereik,b. 6 tot 9 m.,None,1985,Boom niet vrij uitgroeiend,POINT (120154.401 483804.692)
4,919939,Styphnolobium japonicum,Overig,Honingboom,a. tot 6 m.,None,2002,Boom niet vrij uitgroeiend,POINT (120453.549 482894.090)


In [115]:
unqiue = tree_data_oct23['typeObject'].unique()
unqiue

array(['Boom niet vrij uitgroeiend', 'Boom vrij uitgroeiend',
       'Gekandelaberde boom', 'Stobbe', 'Knotboom', 'Vormboom',
       'Fruitboom', None, 'Leiboom', 'Aanwijzingsteken'], dtype=object)

In [116]:
tree_data_oct23['height_of_tree'] = tree_data_oct23['boomhoogte'].replace({
    'b. 6 tot 9 m.': 'b. 6 to 9 m.' ,
    'c. 9 tot 12 m.': 'c. 9 to 12 m.' ,
    'a. tot 6 m.': 'a. to 6 m.' ,
    'q. Niet van toepassing': 'q. Not applicable', 
    'e. 15 tot 18 m.': 'e. 15 to 18 ',
    'd. 12 tot 15 m.': 'd. 12 to 15 m.',
    'f. 18 tot 24 m.': 'f. 18 to 24 m.',
    'g. 24 m. en hoger': 'g. 24 m. and higher'
})

In [117]:
tree_data_oct23['diameter_tree_trunk'] = tree_data_oct23['stamdiamet'].replace({
     '0,2 tot 0,3 m.': "0.2 to 0.3 m.",
     '0,1 tot 0,2 m.': "0.1 to 0.2 m.",
     '0,3 tot 0,5 m.': "0.3 to 0.5 m.",
     '0,5 tot 1 m.': "0.5 to 1 m.",
     '1,0 tot 1,5 m.':"1.0 to 1.5 m.",
     'Onbekend': "unknown",
     '1,5 m. en groter': "1.5 m. and bigger",
     'tot 0,1 m.': "smaller than 0.1 m."
})

In [118]:
tree_data_oct23['object_type'] = tree_data_oct23['typeObject'].replace({
     'Boom niet vrij uitgroeiend': 'non free growing tree',
     'Boom vrij uitgroeiend':'free growing tree',
     'Stobbe':'Tree Stub',
     'Gekandelaberde boom': 'Topped tree',
     'Knotboom': 'Pollarded tree',
     'Vormboom': 'Topiary tree',
     'Fruitboom': 'Fruit tree',
     'Leiboom':'Espalier tree',
     'Bosplantsoen': 'Forest park',
     'Aanwijzingsteken' : 'Indication sign'

})

In [119]:
tree_data_oct23.rename(columns={'soortnaam': 'botanical_name'}, inplace=True)
tree_data_oct23.rename(columns={'SoortnaamN': 'Dutch_name'}, inplace=True)
tree_data_oct23.rename(columns={'jaarVanAan': 'year_planted'}, inplace=True)


In [120]:

tree_data_oct23 = tree_data_oct23.drop(columns=['soortnaamK','boomhoogte','stamdiamet', 'typeObject'])

In [121]:
tree_data_oct23.to_file(output_folder+"tree_data_oct_2023", driver="ESRI Shapefile")

C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_19072\2847698726.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  tree_data_oct23.to_file(output_folder+"tree_data_oct_2023", driver="ESRI Shapefile")
